In [11]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import numpy as np
import cv2
from PIL import Image
import torchvision.transforms as T

In [9]:
device = torch.device('cuda')

In [10]:
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights

weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
model = maskrcnn_resnet50_fpn(weights=weights, progress=False).to(device)
model = model.eval()

In [39]:
# incarc toate clasele din datasetul pe care a fost antrenat modelul
labels = open("coco.names.txt").read().strip().split("\n")
# generez o culoare pentru fiecare clasa in parte
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

In [36]:
image = Image.open("image.jpg")

In [37]:
transforms = weights.transforms()
image = transforms(image)
image=image.unsqueeze(0)
image = image.to(device)

In [38]:
output = model(image) # list of dict
print(len(output)) # equals to how many images that were fed into the model
print(output[0].keys()) # dict_keys(['boxes', 'labels', 'scores', 'masks'])

1
dict_keys(['boxes', 'labels', 'scores', 'masks'])


In [78]:
source = np.asarray(Image.open("image.jpg"))

In [79]:
for index, score in enumerate(output[0]['scores']):
    if score > 0.5:
        source = cv2.rectangle(source, 
                               (int(output[0]['boxes'][index][0]), int(output[0]['boxes'][index][1])), 
                               (int(output[0]['boxes'][index][2]), int(output[0]['boxes'][index][3])),
                               (int(colors[int(output[0]['labels'][index])][0]), int(colors[int(output[0]['labels'][index])][1]), int(colors[int(output[0]['labels'][index])][2])),
                               1)
        source = cv2.putText(source, 
                             labels[int(output[0]['labels'][index])-1], 
                             (int(output[0]['boxes'][index][0]), int(output[0]['boxes'][index][1])-5), 
                             cv2.FONT_HERSHEY_SIMPLEX , 
                             .5, 
                             (int(colors[int(output[0]['labels'][index])][0]), int(colors[int(output[0]['labels'][index])][1]), int(colors[int(output[0]['labels'][index])][2])), 
                             1, 
                             cv2.LINE_AA)

In [80]:
Image.fromarray(source).show()

In [53]:
output[0]['boxes'][0]

tensor([352.9514, 171.5572, 422.1796, 367.7985], device='cuda:0',
       grad_fn=<SelectBackward0>)